In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
import pickle
import torch
import sys
import os
sys.path.append("../")
import time
import numpy as np


/compuworks/anaconda3/envs/infGCN/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# batch_data = pickle.load(open('batch_data_train.pkl', 'rb'))
batch_data = pickle.load(open('mp_batch_data_for_unit_test.pkl', 'rb'))
g, densities, grid_coord, infos = batch_data
batch_data = {
    'atom_types': g.x,
    'atom_coord': g.pos,
    'grid': grid_coord,
    'batch': g.batch,
    'targets': densities,
    'infos': infos
}

In [4]:
for i, info in enumerate(batch_data["infos"]):
    print(f"{i}: {info['file_name']}")

0: mp-979413.chgcar
1: mp-677095.chgcar
2: mp-30510.chgcar
3: mp-1219846.chgcar
4: mp-974432.chgcar
5: mp-1212052.chgcar
6: mp-1039794.chgcar
7: mp-567499.chgcar
8: mp-1047437.chgcar
9: mp-972202.chgcar
10: mp-622018.chgcar
11: mp-1185464.chgcar
12: mp-1209644.chgcar
13: mp-1038594.chgcar
14: mp-1180386.chgcar
15: mp-1209820.chgcar
16: mp-1414188.chgcar
17: mp-5919.chgcar
18: mp-1216550.chgcar
19: mp-10287.chgcar
20: mp-22383.chgcar
21: mp-1218815.chgcar
22: mp-752420.chgcar
23: mp-1186512.chgcar
24: mp-1222421.chgcar
25: mp-1228343.chgcar
26: mp-1226874.chgcar
27: mp-1135.chgcar
28: mp-1025119.chgcar
29: mp-1222993.chgcar
30: mp-20737.chgcar
31: mp-1188110.chgcar


In [5]:
# we will use 12th file, SmIn2Cu9, Site 24, Tetrganoal

In [6]:
batch_data.keys()

dict_keys(['atom_types', 'atom_coord', 'grid', 'batch', 'targets', 'infos'])

In [7]:
atom_types = batch_data["atom_types"]
atom_coord = batch_data["atom_coord"]
grid = batch_data["grid"]
batch = batch_data["batch"]
infos = batch_data["infos"]
targets = batch_data["targets"]

In [8]:
# Refine to 12th file
idx = 12
atom_types = atom_types[batch==idx]
atom_coord = atom_coord[batch==idx]
infos = [infos[idx]]
batch = batch[batch==idx]
batch[batch==idx] = 0
target = targets[idx].unsqueeze(0)
grid = grid[idx].unsqueeze(0)

In [9]:
# test_cell = torch.tensor([[1,0,0.1], [0,1,0], [0.1,0,1]], dtype=torch.float32)
# test_coord = torch.tensor([[0.25, 0.25, 0.25],[0.5,0.25,0.25]], dtype=torch.float32)
# test_batch = torch.tensor([0,0], dtype=torch.int64)
# test_type = torch.tensor([1,2], dtype=torch.int64)

In [10]:
def make_supercell(atom_types, atom_coord, batch, infos, supercell_idx = [1,1,1]):
    cell = infos[0]["cell"]
    supercell_idx = torch.tensor(supercell_idx)
    supercell = torch.einsum("i,ij->ij", supercell_idx.float().to(cell.device),cell)
    super_coords = []
    super_types = []
    for i in range(supercell_idx[0]):
        for j in range(supercell_idx[1]):
            for k in range(supercell_idx[2]):
                super_coords.append(atom_coord + i*cell[0] + j*cell[1] + k*cell[2])
                super_types.append(atom_types)
    
    super_types = torch.cat(super_types, dim=0).to(atom_types.device)
    super_coords = torch.cat(super_coords, dim=0).to(atom_coord.device)
    super_batch = torch.zeros_like(super_types).to(atom_types.device)
    super_infos = infos[0].copy()
    super_infos["cell"] = supercell
    print("number of atoms",len(super_types))
    print("original cell\n", cell)
    print("supercell\n", supercell)
    print("supercell_idx\n", supercell_idx)
    return super_types, super_coords, super_batch, [super_infos]

In [11]:
from source.models.GPWNO_test import GPWNO
from source.baseline.infgcn import InfGCN

In [12]:
def model_helper(num_fourier):
    return GPWNO(
    n_atom_type=85,
    num_radial=16,
    num_spherical=3,
    radial_embed_size=64,
    radial_hidden_size=128,
    num_radial_layer=2,
    num_gcn_layer=2,
    cutoff=3.0,
    is_fc=False,
    gauss_start=0.5,
    gauss_end=5.,
    residual=True,
    pbc=True,
    activation="norm",
    num_fourier=num_fourier,
    num_fourier_time=12,
    width=16,
    fourier_mode=0,
    padding=0,
    use_max_cell=False,
    equivariant_frame=False,
    probe_cutoff=1.5,
    grid_cutoff=0.75,
    normalize=True,
    using_ff=True,
    model_sharing=False,
    num_infgcn_layer=3,
    use_detach=False,
    num_spherical_RNO=3,
    scalar_mask=True,
    mask_cutoff=1.0,
    input_infgcn=False,
    output_positive=False,
    atom_info="/home/holywater2/crystal/GPWNO/atom_info/mp_tetragonal_atom_info.json",
    input_dist=False,
    atomic_gauss_dist=False
).cuda()

In [13]:
model4 = model_helper(4)
model8 = model_helper(8)
model16 = model_helper(16)
model24 = model_helper(24)
model32 = model_helper(32)
model48 = model_helper(48)
model56 = model_helper(56)
model64 = model_helper(64)
model72 = model_helper(72)

/compuworks/anaconda3/envs/infGCN/lib/python3.9/site-packages/pytorch_lightning/utilities/parsing.py:198: Attribute 'criterion' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['criterion'])`.
/compuworks/anaconda3/envs/infGCN/lib/python3.9/site-packages/torch/jit/_check.py:181: UserWarning: The TorchScript type system doesn't support instance-level annotations on empty non-base types in `__init__`. Instead, either 1) use a type annotation in the class body, or 2) wrap the type in `torch.jit.Attribute`.
  warnings.warn("The TorchScript type system doesn't support "


In [14]:
# model2 = InfGCN(
#     n_atom_type=85,
#     num_radial=16,
#     num_spherical=7,
#     radial_embed_size=64,
#     radial_hidden_size=128,
#     num_radial_layer=2,
#     num_gcn_layer=3,
#     cutoff=5.0,
#     is_fc=False,
#     gauss_start=0.5,
#     gauss_end=5.,
#     residual=False,
#     pbc=True,
#     activation="norm",    
# ).cuda()

In [15]:
# model3 = InfGCN(
#     n_atom_type=85,
#     num_radial=16,
#     num_spherical=3,
#     radial_embed_size=64,
#     radial_hidden_size=128,
#     num_radial_layer=2,
#     num_gcn_layer=3,
#     cutoff=5.0,
#     is_fc=False,
#     gauss_start=0.5,
#     gauss_end=5.,
#     residual=False,
#     pbc=True,
#     activation="norm",    
# ).cuda()

In [16]:
def time_stats(model, atom_types, atom_coord, grid, batch,infos, supercell_idx =[1,1,1],num_iter=100,model_type="GPWNO"):
    at, ac, b, ifs = make_supercell(atom_types, atom_coord, batch, infos, supercell_idx)
    g = grid
    t = 0
    probe_degree = 0
    start_time = time.time()
    with torch.no_grad():
        for i in range(num_iter):
            dd = model.forward(at, ac, g, b, ifs)
            if type == "GPWNO":
                t += dd["time"]
            
    end_time = time.time()
    runtime = end_time - start_time
    print("total runtime     :",runtime, runtime/num_iter)
    del at, ac, b, ifs, dd
    return len(atom_types), runtime

In [17]:
from torch_geometric.data.batch import DataBatch

def time_stats_train(model, atom_types, atom_coord, grid, batch,infos, target, supercell_idx =[1,1,1],num_iter=100,model_type="GPWNO"):
    print("mode: train")
    at, ac, b, ifs = make_supercell(atom_types, atom_coord, batch, infos, supercell_idx)
    g = grid
    t = 0
    probe_degree = 0
    databatch = DataBatch(x=at, pos=ac, batch=b)
    # print(databatch)
    start_time = time.time()
    for i in range(num_iter):
        dd = model.training_step([databatch,target,g,ifs],0)
        if type == "GPWNO":
            t += dd["time"]
            
    end_time = time.time()
    runtime = end_time - start_time
    print("total runtime     :",runtime, runtime/num_iter)
    del at, ac, b, ifs, dd
    return len(atom_types), runtime

In [18]:
res_test = np.zeros((7,5))
for i in range(5):
    res = np.zeros((7))
    res[0] += time_stats_train(model4, atom_types, atom_coord, grid, batch, infos, target, supercell_idx = [2,1,1],num_iter=1)[1]
    res[1] += time_stats_train(model8, atom_types, atom_coord, grid, batch, infos, target, supercell_idx = [2,1,1],num_iter=1)[1]
    res[2] += time_stats_train(model16, atom_types, atom_coord, grid, batch, infos, target, supercell_idx = [2,1,1],num_iter=1)[1]
    res[3] += time_stats_train(model24, atom_types, atom_coord, grid, batch, infos, target, supercell_idx = [2,1,1],num_iter=1)[1]
    res[4] += time_stats_train(model32, atom_types, atom_coord, grid, batch, infos, target, supercell_idx = [2,1,1],num_iter=1)[1]
    res[5] += time_stats_train(model48, atom_types, atom_coord, grid, batch, infos, target, supercell_idx = [2,1,1],num_iter=1)[1]
    res[6] += time_stats_train(model64, atom_types, atom_coord, grid, batch, infos, target, supercell_idx = [2,1,1],num_iter=1)[1]
    res_test[:,i] = res

res_test.mean(axis=1), res_test.std(axis=1)

mode: train
number of atoms 48
original cell
 tensor([[0.0000, 0.0000, 5.0236],
        [8.4651, 0.0000, 0.0000],
        [0.0000, 8.4651, 0.0000]], device='cuda:0')
supercell
 tensor([[ 0.0000,  0.0000, 10.0472],
        [ 8.4651,  0.0000,  0.0000],
        [ 0.0000,  8.4651,  0.0000]], device='cuda:0')
supercell_idx
 tensor([2, 1, 1])


/compuworks/anaconda3/envs/infGCN/lib/python3.9/site-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3190.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
/compuworks/anaconda3/envs/infGCN/lib/python3.9/site-packages/pytorch_lightning/core/module.py:420: You are trying to `self.log()` but the `self.trainer` reference is not registered on the model yet. This is most likely because the model hasn't been passed to the `Trainer`


total runtime     : 6.933577060699463 6.933577060699463
mode: train
number of atoms 48
original cell
 tensor([[0.0000, 0.0000, 5.0236],
        [8.4651, 0.0000, 0.0000],
        [0.0000, 8.4651, 0.0000]], device='cuda:0')
supercell
 tensor([[ 0.0000,  0.0000, 10.0472],
        [ 8.4651,  0.0000,  0.0000],
        [ 0.0000,  8.4651,  0.0000]], device='cuda:0')
supercell_idx
 tensor([2, 1, 1])
total runtime     : 2.0723705291748047 2.0723705291748047
mode: train
number of atoms 48
original cell
 tensor([[0.0000, 0.0000, 5.0236],
        [8.4651, 0.0000, 0.0000],
        [0.0000, 8.4651, 0.0000]], device='cuda:0')
supercell
 tensor([[ 0.0000,  0.0000, 10.0472],
        [ 8.4651,  0.0000,  0.0000],
        [ 0.0000,  8.4651,  0.0000]], device='cuda:0')
supercell_idx
 tensor([2, 1, 1])
total runtime     : 0.1923997402191162 0.1923997402191162
mode: train
number of atoms 48
original cell
 tensor([[0.0000, 0.0000, 5.0236],
        [8.4651, 0.0000, 0.0000],
        [0.0000, 8.4651, 0.0000]], d

(array([1.7666976 , 0.54841394, 0.1815743 , 0.19815903, 0.5597867 ,
        0.32400541, 0.41654639]),
 array([2.64792183, 0.77194267, 0.12741472, 0.12975552, 0.72908535,
        0.14436398, 0.12254169]))

In [19]:
res_train = np.zeros((9,5))
for i in range(5):
    res = np.zeros((9))
    res[0] += time_stats_train(model4, atom_types, atom_coord, grid, batch, infos, target, supercell_idx = [2,1,1],num_iter=100)[1]
    res[1] += time_stats_train(model8, atom_types, atom_coord, grid, batch, infos, target, supercell_idx = [2,1,1],num_iter=100)[1]
    res[2] += time_stats_train(model16, atom_types, atom_coord, grid, batch, infos, target, supercell_idx = [2,1,1],num_iter=100)[1]
    res[3] += time_stats_train(model24, atom_types, atom_coord, grid, batch, infos, target, supercell_idx = [2,1,1],num_iter=100)[1]
    res[4] += time_stats_train(model32, atom_types, atom_coord, grid, batch, infos, target, supercell_idx = [2,1,1],num_iter=100)[1]
    res[5] += time_stats_train(model48, atom_types, atom_coord, grid, batch, infos, target, supercell_idx = [2,1,1],num_iter=100)[1]
    res[6] += time_stats_train(model56, atom_types, atom_coord, grid, batch, infos, target, supercell_idx = [2,1,1],num_iter=100)[1]
    res[7] += time_stats_train(model64, atom_types, atom_coord, grid, batch, infos, target, supercell_idx = [2,1,1],num_iter=100)[1]
    res[8] += time_stats_train(model72, atom_types, atom_coord, grid, batch, infos, target, supercell_idx = [2,1,1],num_iter=100)[1]
    res_train[:,i] = res

res_train.mean(axis=1), res_train.std(axis=1)

mode: train
number of atoms 48
original cell
 tensor([[0.0000, 0.0000, 5.0236],
        [8.4651, 0.0000, 0.0000],
        [0.0000, 8.4651, 0.0000]], device='cuda:0')
supercell
 tensor([[ 0.0000,  0.0000, 10.0472],
        [ 8.4651,  0.0000,  0.0000],
        [ 0.0000,  8.4651,  0.0000]], device='cuda:0')
supercell_idx
 tensor([2, 1, 1])
total runtime     : 3.4271621704101562 0.034271621704101564
mode: train
number of atoms 48
original cell
 tensor([[0.0000, 0.0000, 5.0236],
        [8.4651, 0.0000, 0.0000],
        [0.0000, 8.4651, 0.0000]], device='cuda:0')
supercell
 tensor([[ 0.0000,  0.0000, 10.0472],
        [ 8.4651,  0.0000,  0.0000],
        [ 0.0000,  8.4651,  0.0000]], device='cuda:0')
supercell_idx
 tensor([2, 1, 1])
total runtime     : 3.508631706237793 0.03508631706237793
mode: train
number of atoms 48
original cell
 tensor([[0.0000, 0.0000, 5.0236],
        [8.4651, 0.0000, 0.0000],
        [0.0000, 8.4651, 0.0000]], device='cuda:0')
supercell
 tensor([[ 0.0000,  0.0000, 

KeyboardInterrupt: 

In [20]:
res_train = np.zeros((9,5))
for i in range(5):
    res = np.zeros((9))
    res[0] += time_stats(model4, atom_types, atom_coord, grid, batch, infos, supercell_idx = [2,1,1],num_iter=100)[1]
    res[1] += time_stats(model8, atom_types, atom_coord, grid, batch, infos, supercell_idx = [2,1,1],num_iter=100)[1]
    res[2] += time_stats(model16, atom_types, atom_coord, grid, batch, infos, supercell_idx = [2,1,1],num_iter=100)[1]
    res[3] += time_stats(model24, atom_types, atom_coord, grid, batch, infos, supercell_idx = [2,1,1],num_iter=100)[1]
    res[4] += time_stats(model32, atom_types, atom_coord, grid, batch, infos, supercell_idx = [2,1,1],num_iter=100)[1]
    res[5] += time_stats(model48, atom_types, atom_coord, grid, batch, infos, supercell_idx = [2,1,1],num_iter=100)[1]
    res[6] += time_stats(model56, atom_types, atom_coord, grid, batch, infos, supercell_idx = [2,1,1],num_iter=100)[1]
    res[7] += time_stats(model64, atom_types, atom_coord, grid, batch, infos, supercell_idx = [2,1,1],num_iter=100)[1]
    res[8] += time_stats(model72, atom_types, atom_coord, grid, batch, infos, supercell_idx = [2,1,1],num_iter=100)[1]
    res_train[:,i] = res

res_train.mean(axis=1), res_train.std(axis=1)

number of atoms 48
original cell
 tensor([[0.0000, 0.0000, 5.0236],
        [8.4651, 0.0000, 0.0000],
        [0.0000, 8.4651, 0.0000]], device='cuda:0')
supercell
 tensor([[ 0.0000,  0.0000, 10.0472],
        [ 8.4651,  0.0000,  0.0000],
        [ 0.0000,  8.4651,  0.0000]], device='cuda:0')
supercell_idx
 tensor([2, 1, 1])
total runtime     : 2.790585517883301 0.027905855178833008
number of atoms 48
original cell
 tensor([[0.0000, 0.0000, 5.0236],
        [8.4651, 0.0000, 0.0000],
        [0.0000, 8.4651, 0.0000]], device='cuda:0')
supercell
 tensor([[ 0.0000,  0.0000, 10.0472],
        [ 8.4651,  0.0000,  0.0000],
        [ 0.0000,  8.4651,  0.0000]], device='cuda:0')
supercell_idx
 tensor([2, 1, 1])
total runtime     : 2.8599634170532227 0.028599634170532226
number of atoms 48
original cell
 tensor([[0.0000, 0.0000, 5.0236],
        [8.4651, 0.0000, 0.0000],
        [0.0000, 8.4651, 0.0000]], device='cuda:0')
supercell
 tensor([[ 0.0000,  0.0000, 10.0472],
        [ 8.4651,  0.0000

In [21]:
res_inf = np.zeros((7,5))
for i in range(5):
    res = np.zeros((7))
    res[0] += time_stats(model4, atom_types, atom_coord, grid, batch, infos, supercell_idx = [2,1,1],num_iter=100)[1]
    res[1] += time_stats(model8, atom_types, atom_coord, grid, batch, infos, supercell_idx = [2,1,1],num_iter=100)[1]
    res[2] += time_stats(model16, atom_types, atom_coord, grid, batch, infos, supercell_idx = [2,1,1],num_iter=100)[1]
    res[3] += time_stats(model24, atom_types, atom_coord, grid, batch, infos, supercell_idx = [2,1,1],num_iter=100)[1]
    res[4] += time_stats(model32, atom_types, atom_coord, grid, batch, infos, supercell_idx = [2,1,1],num_iter=100)[1]
    res[5] += time_stats(model48, atom_types, atom_coord, grid, batch, infos, supercell_idx = [2,1,1],num_iter=100)[1]
    res[6] += time_stats(model64, atom_types, atom_coord, grid, batch, infos, supercell_idx = [2,1,1],num_iter=100)[1]
    res_inf[:,i] = res

res_inf.mean(axis=1), res_inf.std(axis=1)

number of atoms 48
original cell
 tensor([[0.0000, 0.0000, 5.0236],
        [8.4651, 0.0000, 0.0000],
        [0.0000, 8.4651, 0.0000]], device='cuda:0')
supercell
 tensor([[ 0.0000,  0.0000, 10.0472],
        [ 8.4651,  0.0000,  0.0000],
        [ 0.0000,  8.4651,  0.0000]], device='cuda:0')
supercell_idx
 tensor([2, 1, 1])
total runtime     : 2.748121738433838 0.02748121738433838
number of atoms 48
original cell
 tensor([[0.0000, 0.0000, 5.0236],
        [8.4651, 0.0000, 0.0000],
        [0.0000, 8.4651, 0.0000]], device='cuda:0')
supercell
 tensor([[ 0.0000,  0.0000, 10.0472],
        [ 8.4651,  0.0000,  0.0000],
        [ 0.0000,  8.4651,  0.0000]], device='cuda:0')
supercell_idx
 tensor([2, 1, 1])
total runtime     : 2.8087198734283447 0.028087198734283447
number of atoms 48
original cell
 tensor([[0.0000, 0.0000, 5.0236],
        [8.4651, 0.0000, 0.0000],
        [0.0000, 8.4651, 0.0000]], device='cuda:0')
supercell
 tensor([[ 0.0000,  0.0000, 10.0472],
        [ 8.4651,  0.0000,

(array([ 2.62004447,  2.68919325,  3.75826225,  5.34044099,  5.89663148,
        16.08810048, 28.20457716]),
 array([0.06471088, 0.06046226, 0.10546994, 0.09882588, 0.06993996,
        0.75909942, 0.35775114]))